In [1]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
import math
from six.moves import xrange
from catboost import Pool, CatBoostRegressor

In [3]:
def digitalize(event):
    if event=='view':
        dig=0.1
    elif event=='dislike':
        dig=-10
    elif event=='skip':
        dig=-0.1
    elif event=='like':
        dig=0.5
    return dig

In [4]:
class CustomlossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        # approxes, targets, weights are indexed containers of floats
        # (containers with only __len__ and __getitem__ defined).
        # weights parameter can be None.
        # Returns list of pairs (der1, der2)
        result = []
        for index in xrange(len(targets)):
            der1 = approxes[index]*targets[index]
            if weights is not None:
#            if weights[index] is not None:
                der1 *= weights[index]

            result.append(der1)

        return np.sum(result)

In [5]:
def my_gain(preds, train_data):
    s=0
    for i in range(len(preds)):
        s=s+preds[i]*train_data[i]
    return s

In [6]:
df_stories_react_train=pd.read_csv('stories_reaction_train.csv')
df_customer_train=pd.read_csv('customer_train.csv')
df_stories_description=pd.read_csv('stories_description.csv')
#print(df_stories_react_train.head(5))
#print(df_customer_train.head(5))
#print(df_stories_description.head(5))

In [7]:
df_stories_react_train['event_value']=[digitalize(x) for x in df_stories_react_train['event']]

In [8]:
df1=pd.merge(df_stories_react_train, df_customer_train, how='left', on='customer_id')

In [9]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=11)

In [12]:
X_train=df1.drop(['event', 'event_value','event_dttm','first_session_dttm','job_title'],axis=1)
Y_train=df1['event_value']
print(len(X_train),X_train.head(50))
print(len(Y_train),Y_train.head(50))

473141     customer_id  story_id product_0 product_1 product_2 product_3 product_4  \
0            15       138       NaN       CLS       UTL       UTL       NaN   
1            15       202       NaN       CLS       UTL       UTL       NaN   
2            15       222       NaN       CLS       UTL       UTL       NaN   
3            15       379       NaN       CLS       UTL       UTL       NaN   
4            15       544       NaN       CLS       UTL       UTL       NaN   
5            15       610       NaN       CLS       UTL       UTL       NaN   
6            15       645       NaN       CLS       UTL       UTL       NaN   
7            15       687       NaN       CLS       UTL       UTL       NaN   
8            15       770       NaN       CLS       UTL       UTL       NaN   
9            15       781       NaN       CLS       UTL       UTL       NaN   
10           15       784       NaN       CLS       UTL       UTL       NaN   
11           15       807       NaN       CLS

In [13]:
X_train['product_0'].fillna('NA', inplace=True)
X_train['product_1'].fillna('NA', inplace=True)
X_train['product_2'].fillna('NA', inplace=True)
X_train['product_3'].fillna('NA', inplace=True)
X_train['product_4'].fillna('NA', inplace=True)
X_train['product_5'].fillna('NA', inplace=True)
X_train['product_6'].fillna('NA', inplace=True)
X_train['gender_cd'].fillna('NA', inplace=True)
X_train['marital_status_cd'].fillna('NA', inplace=True)

In [14]:
print
model = CatBoostRegressor(loss_function=CustomlossObjective(), eval_metric='MAE')

In [ ]:
model.fit(X_train, Y_train, cat_features=[2,3,4,5,6,7,8,9,11])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

